<a href="https://colab.research.google.com/github/deebuls/devblog/blob/master/_notebooks/2022-12-19-dirichlet-disentanglement-epistemic-aleatoric-evidential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "Dis-entaglement of Epistemic and Aleatoric uncertainty for Dirichlet Distribution"
> Analysis

- toc: true
- branch: master
- comments: false
- author: Deebul Nair
- categories: [statistics, uncertainty]

## How to separate epistemic and aleatoric uncertaity from Dirichlet distirbution

Also studing the implications of it and proposing the applications of the solution. 

* Formula for [1] .
* theory in [2] 


ToDo : complete the section with info 

[1] Separation of Aleatoric and Epistemic Uncertainty in Deterministic Deep Neural Networks
Denis Huseljic, Bernhard Sick, Marek Herde, Daniel Kottke

[2] Deep Deterministic Uncertainty: A Simple Baseline
Jishnu Mukhoti

In [18]:
import torch
import math 
import numpy as np

In [66]:
 prior = 1
 n_classes = 5
 def predict_epistemic( alpha):
    """Predicts the uncertainty of a sample. (K / alpha_0)"""
    return n_classes * prior / alpha.sum(-1, keepdim=True)

def predict_aleatoric( alpha):
    """Predicts the uncertainty of a sample. (K / alpha_0)"""
   
    proba_in = (alpha / alpha.sum(-1, keepdim=True)).clamp_(1e-8, 1-1e-8)
    entropy = - torch.sum((proba_in * proba_in.log()), dim=-1)
    normalized_entropy = entropy / np.log(n_classes)
    return normalized_entropy

In [67]:
ones = torch.ones(n_classes)
print (predict_epistemic(ones), predict_aleatoric(ones))

tensor([1.]) tensor(1.)


In [68]:
ones / 1.6

tensor([0.6250, 0.6250, 0.6250, 0.6250, 0.6250])

## When alpha of 1 output keeps increasing 

* **Observation** : Both uncertainty reduces 
* **Impact** : When the model puts all confidence on 1 output it shows that the model is confident 

The maximum aleatoric and epistemic uncertitny is both 1

In [69]:
for i in [1, 10, 50, 10000 ]:
  x = torch.ones(n_classes)
  x[0] = i
  print (x)
  print ("Epistemic UE : {}, Aleatoric UE : {}".format(predict_epistemic(x),  predict_aleatoric(x)))
  print ("------------",)

tensor([1., 1., 1., 1., 1.])
Epistemic UE : tensor([1.]), Aleatoric UE : 1.0
------------
tensor([10.,  1.,  1.,  1.,  1.])
Epistemic UE : tensor([0.3571]), Aleatoric UE : 0.6178266406059265
------------
tensor([50.,  1.,  1.,  1.,  1.])
Epistemic UE : tensor([0.0926]), Aleatoric UE : 0.2278686910867691
------------
tensor([1.0000e+04, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00])
Epistemic UE : tensor([0.0005]), Aleatoric UE : 0.002536643762141466
------------


## When all alpha  output keeps increasing 

* **Observation** : Epistemic reduces aleatoric is high  
* **Impact** : When the model puts all confidence on multiple output basically eans that the model is not sure between which .While since some alpha has increased says that its a seen data and theroefore low aleatoric uncertainty

The maximum aleatoric and epistemic uncertainty is both 1

In [70]:
for i in [1, 10, 50, 10000 ]:
  x = torch.ones(n_classes)*i
  print (x)
  print ("Epistemic UE : {}, Aleatoric UE : {}".format(predict_epistemic(x),  predict_aleatoric(x)))
  print ("------------",)

tensor([1., 1., 1., 1., 1.])
Epistemic UE : tensor([1.]), Aleatoric UE : 1.0
------------
tensor([10., 10., 10., 10., 10.])
Epistemic UE : tensor([0.1000]), Aleatoric UE : 1.0
------------
tensor([50., 50., 50., 50., 50.])
Epistemic UE : tensor([0.0200]), Aleatoric UE : 1.0
------------
tensor([10000., 10000., 10000., 10000., 10000.])
Epistemic UE : tensor([1.0000e-04]), Aleatoric UE : 1.0
------------


## Impact of prior 

prior = 50

The highest epistmeic uncertainty increases from 1 to the prior value

In [71]:
#hide 

prior = 50 

for i in [1, 10, 50, 10000 ]:
  x = torch.ones(n_classes)
  x[0] = i
  print (x)
  print ("Epistemic UE : {}, Aleatoric UE : {}".format(predict_epistemic(x),  predict_aleatoric(x)))
  print ("------------",)

for i in [1, 10, 50, 10000 ]:
  x = torch.ones(n_classes)*i
  print (x)
  print ("Epistemic UE : {}, Aleatoric UE : {}".format(predict_epistemic(x),  predict_aleatoric(x)))
  print ("------------",)

tensor([1., 1., 1., 1., 1.])
Epistemic UE : tensor([50.]), Aleatoric UE : 1.0
------------
tensor([10.,  1.,  1.,  1.,  1.])
Epistemic UE : tensor([17.8571]), Aleatoric UE : 0.6178266406059265
------------
tensor([50.,  1.,  1.,  1.,  1.])
Epistemic UE : tensor([4.6296]), Aleatoric UE : 0.2278686910867691
------------
tensor([1.0000e+04, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00])
Epistemic UE : tensor([0.0250]), Aleatoric UE : 0.002536643762141466
------------
tensor([1., 1., 1., 1., 1.])
Epistemic UE : tensor([50.]), Aleatoric UE : 1.0
------------
tensor([10., 10., 10., 10., 10.])
Epistemic UE : tensor([5.]), Aleatoric UE : 1.0
------------
tensor([50., 50., 50., 50., 50.])
Epistemic UE : tensor([1.]), Aleatoric UE : 1.0
------------
tensor([10000., 10000., 10000., 10000., 10000.])
Epistemic UE : tensor([0.0050]), Aleatoric UE : 1.0
------------


## Conclusions

* Dirichlet distirbution can be dis-entagled with aleatoric and epistemic uncertainty.
* When all alpha is 1 - both uncertainty are also 1 impling that the network doesnt know anything
* If only 1 alpha is higher then both uncertainty is high
* The higher the alpha the lower both the uncertainty
* If multiple alpha is higher then only aleatoric is high epistemic stays low. Impling that since the some alpha was increased the network has seen the input and its not sure which amongst the outputs are correct.


### Use Case
#### 1. For identifying OOD data
1. For the training dataset measure the epistmic uncertainty of the correct predictions. It should be less than 1 and near to zero
2. During prediction if epistimeic Ue is higher than the training max then that data should be considered OOD and handled appropriately

#### 2. For handling indomain uncertain data
1. If the epistemic unertainty is is range but he aleatoric is high we can use these in embodied situation to collect the data(image) from different view and either fuse and make decision. Example if blur image - then differ to predict but dont flag as OOD just wit for nxt better data

